# Init

In [4]:
from data_preprocess import read_and_normalize_train_data, read_and_normalize_test_data2, save_submission
import numpy as np
import time
from vgg16_run import save_model, read_model

/home/shauk/.conda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/shauk/.conda/envs/tensorflow/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
img_rows, img_cols = 224, 224
batch_size = 32
nb_epoch = 10

In [6]:
import tensorflow as tf
from keras.utils import plot_model
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator

In [7]:
def resnet50_model():
    base_model = ResNet50(include_top=False, input_shape=(224,224,3))
    resnet50_output = base_model.output
    predictions = Dense(10, activation='softmax')(Flatten()(resnet50_output))
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
from sklearn.model_selection import KFold

In [9]:
model_name='resnet50'
nfolds = 10
kf = KFold(n_splits=nfolds, shuffle=True, random_state=42)

# Train use ImageDataGenerator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('data/train', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 17943 images belonging to 10 classes.


In [8]:
validate_datagen = ImageDataGenerator(rescale=1./255)
validate_generator = train_datagen.flow_from_directory('data/validation', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 4481 images belonging to 10 classes.


In [9]:
model_name = 'resnet50'
model = resnet50_model()
model.fit_generator(train_generator, steps_per_epoch=2000, epochs=20, validation_data=validate_generator, validation_steps=800)
save_model(model, 100, model_name)

Epoch 1/20
2000/2000 [==============================] - 616s 308ms/step - loss: 2.0387 - acc: 0.2997 - val_loss: 1.8525 - val_acc: 0.4361
Epoch 2/20
2000/2000 [==============================] - 612s 306ms/step - loss: 1.7180 - acc: 0.4654 - val_loss: 1.5943 - val_acc: 0.5605
Epoch 3/20
2000/2000 [==============================] - 614s 307ms/step - loss: 1.5295 - acc: 0.5448 - val_loss: 1.4655 - val_acc: 0.5434
Epoch 4/20
2000/2000 [==============================] - 614s 307ms/step - loss: 1.3941 - acc: 0.5976 - val_loss: 1.3372 - val_acc: 0.6203
Epoch 5/20
2000/2000 [==============================] - 617s 308ms/step - loss: 1.2831 - acc: 0.6392 - val_loss: 1.2436 - val_acc: 0.6642
Epoch 6/20
2000/2000 [==============================] - 615s 307ms/step - loss: 1.1987 - acc: 0.6733 - val_loss: 1.1597 - val_acc: 0.6977
Epoch 7/20
2000/2000 [==============================] - 620s 310ms/step - loss: 1.1217 - acc: 0.7002 - val_loss: 1.0843 - val_acc: 0.7358
Epoch 8/20
2000/2000 [============

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('data/test', target_size=(224,224), batch_size=batch_size)

Found 79726 images belonging to 1 classes.


In [11]:
test_ids = test_generator.filenames
test_ids = [d.strip('c0/') for d in test_ids]

In [12]:
#model = read_model(100, model_name)
test_prediction = model.predict_generator(test_generator, verbose=1)
save_submission(test_ids, test_prediction)

2492/2492 [==============================] - 552s 222ms/step


# Resnet50 with 2 layer fc

In [10]:
def resnet50_model_02():
    base_model = ResNet50(include_top=False, input_shape=(224,224,3))
    resnet50_output = base_model.output
    fc1 = Dense(4096, activation='softmax')(Flatten()(resnet50_output))
    predictions = Dense(10, activation='softmax')(fc1)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model 

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('data/train', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 17943 images belonging to 10 classes.


In [12]:
validate_datagen = ImageDataGenerator(rescale=1./255)
validate_generator = validate_datagen.flow_from_directory('data/validation', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 4481 images belonging to 10 classes.


In [13]:
model_name = 'resnet50_with_2_fc'
model = resnet50_model()
model.fit_generator(train_generator, steps_per_epoch=2000, epochs=5, validation_data=validate_generator, validation_steps=800)

Epoch 1/5
2000/2000 [==============================] - 612s 306ms/step - loss: 2.0393 - acc: 0.3018 - val_loss: 1.8265 - val_acc: 0.4533
Epoch 2/5
2000/2000 [==============================] - 608s 304ms/step - loss: 1.7193 - acc: 0.4630 - val_loss: 1.6177 - val_acc: 0.5091
Epoch 3/5
2000/2000 [==============================] - 608s 304ms/step - loss: 1.5288 - acc: 0.5428 - val_loss: 1.4428 - val_acc: 0.5927
Epoch 4/5
2000/2000 [==============================] - 610s 305ms/step - loss: 1.3908 - acc: 0.6001 - val_loss: 1.3353 - val_acc: 0.6427
Epoch 5/5
2000/2000 [==============================] - 610s 305ms/step - loss: 1.2841 - acc: 0.6404 - val_loss: 1.2342 - val_acc: 0.6648


In [14]:
save_model(model, 5, 'resnet50_with2_fc')

Found 79726 images belonging to 1 classes.
2492/2492 [==============================] - 540s 217ms/step


 # Image Generator with Preprocessing Procedure

In [17]:
#train_datagen = ImageDataGenerator(rescale=1./255, )
train_datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=(0.85, 1.1),
    channel_shift_range=10.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False)
train_generator = train_datagen.flow_from_directory('data/train', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 17943 images belonging to 10 classes.


In [19]:
validate_datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=(0.85, 1.1),
    channel_shift_range=10.,
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=False,
    vertical_flip=False)
validate_generator = validate_datagen.flow_from_directory('data/validation', target_size=(224,224), batch_size=batch_size, class_mode='categorical')

Found 4481 images belonging to 10 classes.


In [20]:
model_name = 'resnet50_with_2_fc'
model = resnet50_model()
model.fit_generator(train_generator, steps_per_epoch=2000, epochs=5, validation_data=validate_generator, validation_steps=800)

Epoch 1/5
2000/2000 [==============================] - 1167s 583ms/step - loss: 0.7724 - acc: 0.7445 - val_loss: 0.4284 - val_acc: 0.8623
Epoch 2/5
2000/2000 [==============================] - 1156s 578ms/step - loss: 0.3517 - acc: 0.8869 - val_loss: 0.2969 - val_acc: 0.9044
Epoch 3/5
2000/2000 [==============================] - 1156s 578ms/step - loss: 0.2645 - acc: 0.9156 - val_loss: 0.2125 - val_acc: 0.9354
Epoch 4/5
2000/2000 [==============================] - 1160s 580ms/step - loss: 0.2178 - acc: 0.9312 - val_loss: 0.2328 - val_acc: 0.9250
Epoch 5/5
2000/2000 [==============================] - 1156s 578ms/step - loss: 0.1925 - acc: 0.9378 - val_loss: 0.1783 - val_acc: 0.9456


In [24]:
test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory('data/test', target_size=(224,224), batch_size=batch_size, class_mode=None, shuffle=False)
test_ids = test_generator.filenames
test_ids = [d.strip('c0/') for d in test_ids]
test_prediction = model.predict_generator(test_generator, verbose=1)
save_submission(test_ids, test_prediction)

Found 79726 images belonging to 1 classes.
2492/2492 [==============================] - 542s 218ms/step


In [25]:
test_ids = [d.strip('c0/') for d in test_ids]
save_submission(test_ids, test_prediction)